# Bsky integration

In [ ]:
//| export

import { BskyXRPC } from "@mary/bluesky-client";
import type {
  AppBskyEmbedImages,
  AppBskyFeedDefs,
  AppBskyFeedPost,
} from "@mary/bluesky-client/lexicons";

type Thread = AppBskyFeedDefs.ThreadViewPost;
type Post = AppBskyFeedDefs.PostView;
type PostRecord = AppBskyFeedPost.Record;
type EmbedImages = AppBskyEmbedImages.View;

const rpc = new BskyXRPC({ service: "https://public.api.bsky.app" });

# Post URL to @at protocol URI

Figure out how to convert bsky.app URL to a URI we can use to fetch data. We
need to go from this:

```
https://bsky.app/profile/callmephilip.com/post/3lcskn64bss2d
```

To smth like this

```
at://did:plc:ubdeopbbkbgedccgbum7dhsh/app.bsky.feed.post/3lcskn64bss2d"
```

`did:plc:ubdeopbbkbgedccgbum7dhsh` is DID for callmephilip.com handle, which
needs to be resolved first

In [ ]:
//| export

const postURLToAtpURI = async (
  postUrl: string,
): Promise<string> => {
  const urlParts = new URL(postUrl);
  const pathParts = urlParts.pathname.split("/");
  const h = await rpc.get("com.atproto.identity.resolveHandle", {
    params: {
      handle: pathParts[2],
    },
  });

  return `at://${h.data.did}/app.bsky.feed.post/${pathParts[4]}`;
};

In [ ]:
await postURLToAtpURI(
  "https://bsky.app/profile/callmephilip.com/post/3lcskn64bss2d",
);

"at://did:plc:ubdeopbbkbgedccgbum7dhsh/app.bsky.feed.post/3lcskn64bss2d"

# Grabbing thread data

Thread has a bunch of nested posts inside replies. Unwrap this into a list of
posts

In [ ]:
//| export

const unwrapThreadPosts = (
  thread: Thread,
): Post[] => {
  const posts: Post[] = [];

  // Add root post
  if (thread.post) {
    posts.push(thread.post);
  }

  // Recursively handle replies
  if (thread.replies) {
    thread.replies.forEach((reply) => {
      posts.push(...unwrapThreadPosts(reply as Thread));
    });
  }

  // Handle nested reply if present
  if ("parent" in thread && thread.parent) {
    posts.push(
      ...unwrapThreadPosts(thread.parent as Thread),
    );
  }

  return posts;
};

export const downloadThread = async (
  postUrl: string,
) => {
  const d = await rpc.get("app.bsky.feed.getPostThread", {
    params: {
      uri: await postURLToAtpURI(postUrl),
    },
  });
  const r = unwrapThreadPosts(d.data.thread as Thread);
  return r.sort(
    (a: Post, b: Post) =>
      new Date((a.record as PostRecord).createdAt).getTime() -
      new Date((b.record as PostRecord).createdAt).getTime(),
  );
};

In [ ]:
const posts = await downloadThread(
  // "https://bsky.app/profile/callmephilip.com/post/3lcskn64bss2d",
  "https://bsky.app/profile/xenova.bsky.social/post/3ldlswuvnxs2i",
);

# Converting posts to MD

The most basic setup is when there is just a piece of text display. More complex
cases will have links, attachments

In [ ]:
//| export

// from https://github.com/mary-ext/skeetdeck/blob/aa0cb74c0ace489b79d2671c4b9e740ec21623c7/app/api/richtext/unicode.ts

const encoder = new TextEncoder();
const decoder = new TextDecoder();

interface UtfString {
  u16: string;
  u8: Uint8Array;
}

const createUtfString = (utf16: string): UtfString => {
  return {
    u16: utf16,
    u8: encoder.encode(utf16),
  };
};

// const getUtf8Length = (utf: UtfString) => {
// 	return utf.u8.byteLength;
// };

const sliceUtf8 = (utf: UtfString, start?: number, end?: number) => {
  return decoder.decode(utf.u8.slice(start, end));
};

In [ ]:
//| export

export const postToMd = (post: Post): string => {
  const record = post.record as PostRecord;
  const text = record.text;
  let richtext = text;
  let embeds = "";

  console.log(">>>>>>>>>>> text is", text);

  if (record.facets) {
    for (const facet of record.facets) {
      for (const feature of facet.features) {
        if (feature.$type === "app.bsky.richtext.facet#link") {
          const linkPlaceholder = sliceUtf8(
            createUtfString(text),
            facet.index.byteStart,
            facet.index.byteEnd,
          );
          richtext = richtext.replace(
            linkPlaceholder,
            `[${linkPlaceholder}](${feature.uri})`,
          );
        }
      }
    }
  }

  if (post.embed && post.embed.$type === "app.bsky.embed.images#view") {
    const e = post.embed as EmbedImages;
    for (const image of e.images) {
      embeds += `![${
        image.alt || "no image description"
      }](${image.fullsize})\n`;
    }
  }

  const [d, t] = record.createdAt.split("T");
  const [h, m] = t.split(":");

  console.log(">>>>", "lines", richtext.split("\n"));

  return [
    `> [${post.author.displayName} - @${post.author.handle}](https://bsky.app/profile/${post.author.handle}) **${d} ${
      [h, m].join(
        ":",
      )
    }**`,
    ...richtext.split("\n").filter((l: string) => l.trim() !== ""),
    embeds,
  ].join("\n\n");
};

// await Deno.jupyter.display(
//   {
//     "text/markdown": postToMd(posts[1]),
//   },
//   { raw: true }
// );

In [ ]:
await Deno.jupyter.display(
  {
    "text/markdown": postToMd(posts[0]),
  },
  { raw: true },
);

>>>>>>>>>>> text is Introducing Moonshine Web: real-time speech recognition running 100% locally in your browser!
🚀 Faster and more accurate than Whisper
🔒 Privacy-focused (no data leaves your device)
⚡️ WebGPU accelerated (w/ WASM fallback)
🔥 Powered by ONNX Runtime Web and Transformers.js

Demo + source code below! 👇


>>>> lines [
  "Introducing Moonshine Web: real-time speech recognition running 100% locally in your browser!",
  "🚀 Faster and more accurate than Whisper",
  "🔒 Privacy-focused (no data leaves your device)",
  "⚡️ WebGPU accelerated (w/ WASM fallback)",
  "🔥 Powered by ONNX Runtime Web and Transformers.js",
  "",
  "Demo + source code below! 👇"
]


> [Xenova - @xenova.bsky.social](https://bsky.app/profile/xenova.bsky.social) **2024-12-18 16:51**

Introducing Moonshine Web: real-time speech recognition running 100% locally in your browser!

🚀 Faster and more accurate than Whisper

🔒 Privacy-focused (no data leaves your device)

⚡️ WebGPU accelerated (w/ WASM fallback)

🔥 Powered by ONNX Runtime Web and Transformers.js

Demo + source code below! 👇



Support links inside posts

In [ ]:
await Deno.jupyter.display(
  {
    "text/markdown": postToMd(posts[0]),
  },
  { raw: true },
);

>>>>>>>>>>> text is Introducing Moonshine Web: real-time speech recognition running 100% locally in your browser!
🚀 Faster and more accurate than Whisper
🔒 Privacy-focused (no data leaves your device)
⚡️ WebGPU accelerated (w/ WASM fallback)
🔥 Powered by ONNX Runtime Web and Transformers.js

Demo + source code below! 👇


>>>> lines [
  "Introducing Moonshine Web: real-time speech recognition running 100% locally in your browser!",
  "🚀 Faster and more accurate than Whisper",
  "🔒 Privacy-focused (no data leaves your device)",
  "⚡️ WebGPU accelerated (w/ WASM fallback)",
  "🔥 Powered by ONNX Runtime Web and Transformers.js",
  "",
  "Demo + source code below! 👇"
]


> [Xenova - @xenova.bsky.social](https://bsky.app/profile/xenova.bsky.social) **2024-12-18 16:51**

Introducing Moonshine Web: real-time speech recognition running 100% locally in your browser!

🚀 Faster and more accurate than Whisper

🔒 Privacy-focused (no data leaves your device)

⚡️ WebGPU accelerated (w/ WASM fallback)

🔥 Powered by ONNX Runtime Web and Transformers.js

Demo + source code below! 👇



In [ ]:
await Deno.jupyter.display(
  {
    "text/markdown": postToMd(posts[1]),
  },
  { raw: true },
);

>>>>>>>>>>> text is Huge shout-out to the Useful Sensors team for such an amazing model and to Wael Yasmina for his 3D audio visualizer tutorial! 🤗

‍💻 Source code: github.com/huggingface/...
🔗 Online demo: huggingface.co/spaces/webml...


>>>> lines [
  "Huge shout-out to the Useful Sensors team for such an amazing model and to Wael Yasmina for his 3D audio visualizer tutorial! 🤗",
  "",
  "‍💻 Source code: [github.com/huggingface/...](https://github.com/huggingface/transformers.js-examples/tree/main/moonshine-web)",
  "🔗 Online demo: [huggingface.co/spaces/webml...](https://huggingface.co/spaces/webml-community/moonshine-web)"
]


> [Xenova - @xenova.bsky.social](https://bsky.app/profile/xenova.bsky.social) **2024-12-18 16:51**

Huge shout-out to the Useful Sensors team for such an amazing model and to Wael Yasmina for his 3D audio visualizer tutorial! 🤗

‍💻 Source code: [github.com/huggingface/...](https://github.com/huggingface/transformers.js-examples/tree/main/moonshine-web)

🔗 Online demo: [huggingface.co/spaces/webml...](https://huggingface.co/spaces/webml-community/moonshine-web)



In [ ]:
//| export

export const downloadPostToMd = async (postUrl: string): Promise<string> => {
  const posts = await downloadThread(postUrl);

  return posts.map((post) => postToMd(post)).join("\n\n");
};

In [ ]:
await Deno.jupyter.display(
  {
    "text/markdown": await downloadPostToMd(
      "https://bsky.app/profile/callmephilip.com/post/3lcskn64bss2d",
    ),
  },
  { raw: true },
);

>>>>>>>>>>> text is 🧵 Notes on Deno + Jupyter 🦕 🔭


>>>> lines [ "🧵 Notes on Deno + Jupyter 🦕 🔭" ]


>>>>>>>>>>> text is Jupyter kernel for Deno - Deno docs docs.deno.com/runtime/refe...


>>>> lines [
  "Jupyter kernel for Deno - Deno docs [docs.deno.com/runtime/refe...](https://docs.deno.com/runtime/reference/cli/jupyter/)"
]


>>>>>>>>>>> text is Repro of original Deno2 jupyter demo with broken mermaid integration - gist.github.com/aaronblondea...


...

>>>>>>>>>>> text is As far as tests are concerned, ended up dumping them into a separate ts module and then using native `deno test`


>>>> lines [
  "As far as tests are concerned, ended up dumping them into a separate ts module and then using native `deno test`"
]


>>>>>>>>>>> text is useful reference - PR with Jupyter support for Deno github.com/denoland/den...


>>>> lines [
  "useful reference - PR with Jupyter support for Deno [github.com/denoland/den...](https://github.com/denoland/deno/pull/20337)"
]


> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2024-12-08 15:46**

🧵 Notes on Deno + Jupyter 🦕 🔭



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2024-12-08 15:49**

Jupyter kernel for Deno - Deno docs [docs.deno.com/runtime/refe...](https://docs.deno.com/runtime/reference/cli/jupyter/)



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2024-12-08 15:56**

Repro of original Deno2 jupyter demo with broken mermaid integration - [gist.github.com/aaronblondea...](https://gist.github.com/aaronblondeau/02f94256aab61f409d1f820cc80ea571)



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2024-12-08 16:00**

Rich displays for jupyter js kernels - [deno.land/x/display@v1...](https://deno.land/x/display@v1.1.2)

![no image description](https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:ubdeopbbkbgedccgbum7dhsh/bafkreidfehjdftii7yrynrzeecpqxqjycabduk2yasxeeze5zi3dji4cre@jpeg)


> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2024-12-08 16:26**

@kylekelley.bsky.social has a bunch of Deno in notebook examples here - [github.com/rgbkrk/denot...](https://github.com/rgbkrk/denotebooks)



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2024-12-08 16:44**

nice video showing deno + jupyter (via noteable RIP) [www.youtube.com/watch?v=b5OK...](https://www.youtube.com/watch?v=b5OKkMuue3Q)



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2024-12-08 16:53**

anywidget for deno jupyter kernel - [jsr.io/@anywidget/d...](https://jsr.io/@anywidget/deno)



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2024-12-16 18:03**

Hit 2 issues with Deno + Jupyter: running notebooks from cmd and tracking failing tests.



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2024-12-16 18:06**

notebook cmd line issue is described here: [github.com/jupyter/nbcl...](https://github.com/jupyter/nbclient/issues/321). Includes a PR with a suggested fix. In the meantime, using some homemade stuff based on the nbclient fork - [github.com/callmephilip...](https://github.com/callmephilip/jurassic/blob/main/.jurassic/runnb.py)



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2024-12-16 18:07**

As far as tests are concerned, ended up dumping them into a separate ts module and then using native `deno test`



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2024-12-16 20:24**

useful reference - PR with Jupyter support for Deno [github.com/denoland/den...](https://github.com/denoland/deno/pull/20337)

